In [ ]:
! pip install hazm
from __future__ import unicode_literals
import pandas as pd
import pickle
import numpy as np
from hazm import Normalizer, Stemmer, word_tokenize, sent_tokenize
from scipy import sparse
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MinMaxScaler
import json
import gensim 
from gensim.models import Word2Vec
from keras import Sequential
from keras.layers import Dense, BatchNormalization
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC, SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
import tensorflow as tf
from keras.models import Model
from keras import backend as K
from keras.layers import Input, RNN, GRU, LSTM, Lambda, concatenate, Dense, Embedding, Dropout, Activation, Conv1D, MaxPooling1D, Flatten, Multiply, GlobalMaxPooling1D, TimeDistributed, SpatialDropout1D, AveragePooling1D, GlobalAveragePooling1D
from keras.initializers import Constant
from keras.preprocessing.sequence import pad_sequences
from keras.utils import Sequence
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import zipfile

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
class data_utils:
    
    def __init__(self, texts, labels):
        
        self.texts = texts
        self.labels = labels
        self.word_vec = {}
        self.vocab_dict = {}
        self.vec_dim = 0
        self.label_dict = {}
        count = 0
        for l in labels:
            if l not in self.label_dict:
                self.label_dict[l] = count
                count += 1
                
    def create_embeddings(self, dimension=100):
        
        self.vec_dim = dimension
        data = []
        for d in self.texts:
            for i in sent_tokenize(d): 
                temp = [] 
                for j in word_tokenize(i): 
                    temp.append(j) 

                data.append(temp)

        model1 = gensim.models.Word2Vec(data, min_count = 1,  
                                      size = dimension, window = 5)
        model1.train(data, total_examples=len(data), epochs=30)
        self.word_vec = {}
        for w in model1.wv.vocab:
            self.word_vec[w] = model1.wv.get_vector(w)
    
            
    def create_vocab(self, vocab_size):
        
        train_tok_sents = list(map(word_tokenize, self.texts))
        special_toks = ['<pad>','<unk>']
        c = Counter()
        for s in train_tok_sents:
            c.update(s)

        words = [w for w,_ in c.most_common(vocab_size)] 
        vocab = special_toks + words
        self.vocab_dict = {w:i for i, w in enumerate(vocab)}
    
    def create_embedding_matrix(self):
        
        mean_embed = np.mean(np.array(list(self.word_vec.values())), axis=0)
        embedding_matrix = np.zeros((len(self.vocab_dict), self.vec_dim))
        for word, i in self.vocab_dict.items():
            embedding_matrix[i] = self.word_vec.get(word, mean_embed)
            
        embedding_matrix[0] = np.zeros(shape=(self.vec_dim, ))
        return embedding_matrix
        

    

In [ ]:
import matplotlib.pyplot as plt

def visualize_loss_and_acc(history):
  history_dict = history.history
  loss_values = history_dict['loss']
  val_loss_values = history_dict['val_loss']
  acc = history_dict['acc']

  epochs = range(1, len(acc) + 1)

  f = plt.figure(figsize=(10,3))

  plt.subplot(1,2,1)
  plt.plot(epochs, loss_values, 'bo', label='Training loss')
  plt.plot(epochs, val_loss_values, 'b', label='Validation loss')
  plt.title('Training and validation loss')
  plt.xlabel('Epochs')
  plt.ylabel('Loss')
  plt.legend()


  acc_values = history_dict['acc']
  val_acc = history_dict['val_acc']

  plt.subplot(1,2,2)
  plt.plot(epochs, acc, 'bo', label='Training acc')
  plt.plot(epochs, val_acc, 'b', label='Validation acc')
  plt.title('Training and validation accuracy')
  plt.xlabel('Epochs')
  plt.ylabel('accuracy')
  plt.legend()

  plt.show()

In [ ]:
# zip_ref = zipfile.ZipFile('/content/drive/My Drive/file.zip', 'r')
zip_ref = zipfile.ZipFile('./file.zip', 'r')
zip_ref.extractall('.')
zip_ref.close()

In [ ]:
texts, labels = pickle.load(open('./file', 'rb'))

In [ ]:
train_sents, test_sents, train_labels, test_labels = train_test_split(texts, labels, stratify=labels,)

In [ ]:
du = data_utils(train_sents, train_labels)

In [ ]:
du.create_embeddings(dimension=100)

In [ ]:
du.create_vocab(50000)

# SVM with BOW+tfi-df

In [ ]:
NGRAM_RANGE = 3
for feature in ['tfidf', True, False]:
    
    vec = TfidfVectorizer(ngram_range=(1,NGRAM_RANGE)) if feature=='tfidf' else CountVectorizer(binary=feature, ngram_range=(1,NGRAM_RANGE))
    clf = LinearSVC(max_iter=100)
    pipeline = Pipeline([('vectorizer', vec), ('classifier', clf)])
    pipeline.fit(train_sents, train_labels)
    feature_name = 'tfidf'
    if feature != 'tfidf':
        feature_name = 'binary' if feature else 'count'
    print('accuracy using {} for feature values'.format(feature_name))
    print(accuracy_score(test_labels, pipeline.predict(test_sents)))

# ANN with average embeddings

In [ ]:
def create_nn(embedding_matrix, num_classes):

    i = Input((None,))
  
    embedding_layer = Embedding(
      input_dim=len(du.vocab_dict)+2, 
      output_dim=du.vec_dim,
      embeddings_initializer=Constant(embedding_matrix),
      mask_zero=True,
    )
    reduce_sumer=Lambda(lambda x: tf.reduce_sum(x, 1) )
    nz_counter = Lambda(lambda x: tf.to_float(tf.count_nonzero(x, 1, keepdims=True)))
    division = Lambda(lambda x: tf.div(x[0], x[1]))
    inp = embedding_layer(i)
    inp = reduce_sumer(inp)
    nz = nz_counter(i)
    inp = division([inp, nz])
  
  
    y = Dense(1024, activation='relu', input_shape=(du.vec_dim,))(inp)
    y = BatchNormalization()(y)
    y = Dropout(0.2)(y)
    y = Dense(512, activation='relu')(y)
    y = BatchNormalization()(y)
    y = Dropout(0.2)(y)
    y = Dense(256, activation='relu')(y)
    y = BatchNormalization()(y)
    y = Dropout(0.2)(y)
    y = Dense(num_classes, activation='softmax')(y)
  
    model = Model(inputs=i, outputs=y)
  
    return model

In [ ]:
MAX_LEN = 200
train_tok_sents = list(map(word_tokenize, train_sents))
test_tok_sents = list(map(word_tokenize, test_sents))
embedding_matrix = du.create_embedding_matrix()
unk_id = du.vocab_dict['<unk>']
x_train = pad_sequences([[du.vocab_dict.get(w, unk_id) for w in s] for s in train_tok_sents],
                        padding='post', truncating='post', maxlen=MAX_LEN)
x_test = pad_sequences([[du.vocab_dict.get(w, unk_id) for w in s] for s in test_tok_sents],
                       padding='post', truncating='post', maxlen=MAX_LEN)
y_train = to_categorical([du.label_dict[l] for l in train_labels])
y_test = to_categorical([du.label_dict[l] for l in test_labels])

In [ ]:
nn = create_nn(embedding_matrix, len(du.label_dict))
nn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
nn.summary()

In [ ]:
nn.fit(x_train, y_train,epochs=15, validation_data=(x_test, y_test) ,batch_size=256, verbose=1)

# RNN methods

In [ ]:
BATCH_SIZE=256

In [ ]:
class VariableLengthGenerator(Sequence):

    def __init__(self, X, Y, batch_size, num_classes, shuffle=False):

    
        self.x = np.array(X)
        self.y = np.array(Y)
        self.batch_size = batch_size
        self.shuffle = shuffle
  
    def __len__(self):
    
        return int(np.ceil(len(self.x)/self.batch_size))
  
    def __getitem__(self, idx):

        x_batch = self.x[idx*self.batch_size: (idx+1)*self.batch_size]
        y_batch = self.y[idx*self.batch_size: (idx+1)*self.batch_size]
        if len(x_batch) != self.batch_size:
            x_batch = self.x[len(self.x)-self.batch_size: len(self.x)]
            y_batch = self.y[len(self.y)-self.batch_size: len(self.y)]

        x = pad_sequences(x_batch, padding='post')
        y = y_batch
    
        return (x, y)
  
    def on_epoch_end(self):

        if self.shuffle:
            shuffled_indexes = list(range(len(self.x)))
            np.random.shuffle(shuffled_indexes)
            self.x, self.y = self.x[shuffled_indexes], self.y[shuffled_indexes]

In [ ]:
def get_simple_bilstm(embedding_matrix, num_classes, num_layers, hidden_state_size):
    
    x = Input((None,))
    embedding_layer = Embedding(
      input_dim=len(du.vocab_dict)+2, 
      output_dim=du.vec_dim,
      embeddings_initializer=Constant(embedding_matrix),
      mask_zero=True,
    )
    
    inp = embedding_layer(x)
    
    for i in range(num_layers-1):
    
        fw_rnn = LSTM(units=hidden_state_size, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)(inp)
        bw_rnn = LSTM(units=hidden_state_size, return_sequences=True, go_backwards=True, dropout=0.2, recurrent_dropout=0.2)(inp)
        inp = concatenate([fw_rnn, bw_rnn], axis=-1) 
        
    out_fw_rnn = LSTM(units=hidden_state_size, return_sequences=False)(inp)
    out_bw_rnn = LSTM(units=hidden_state_size, return_sequences=False, go_backwards=True)(inp)
    out_rnn = concatenate([out_fw_rnn, out_bw_rnn], axis=-1)
    
    y = Dense(256, activation='relu', input_shape=(hidden_state_size*2,))(out_rnn)
    y = BatchNormalization()(y)
    y = Dropout(0.3)(y)
    y = Dense(128, activation='relu')(y)
    y = BatchNormalization()(y)
    y = Dropout(0.3)(y)
    y = Dense(num_classes, activation='softmax')(y)
    
    model = Model(inputs=x, outputs=y)
    return model

In [ ]:
def get_att_bilstm(embedding_matrix, num_classes, num_layers, hidden_state_size, att_layer_size=[]):
    
    x = Input((None,))
    embedding_layer = Embedding(
      input_dim=len(du.vocab_dict)+2, 
      output_dim=du.vec_dim,
      embeddings_initializer=Constant(embedding_matrix),
      mask_zero=True,
    )
    
    inp = embedding_layer(x)
    
    for i in range(num_layers):
    
        fw_rnn = LSTM(units=hidden_state_size, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)(inp)
        bw_rnn = LSTM(units=hidden_state_size, return_sequences=True, go_backwards=True, dropout=0.2, recurrent_dropout=0.2)(inp)
        inp = concatenate([fw_rnn, bw_rnn], axis=-1) 
    
    alpha = inp
    alpha = Lambda(lambda x: tf.math.tanh(x))(alpha)
    for l in att_layer_size:
      alpha = TimeDistributed(Dense(l, activation='relu'))(alpha)
    alpha = TimeDistributed(Dense(1, activation='relu'))(alpha)
    alpha = Lambda(lambda x: tf.reshape(x, [-1, tf.shape(x)[1]]))(alpha)
    alpha = Lambda(lambda x: tf.nn.softmax(x))(alpha)
    alpha = Lambda(lambda x: tf.reshape(x, [-1, tf.shape(x)[1], 1]))(alpha)
    att_out = Lambda(lambda x: tf.math.multiply(x[0], x[1]))([alpha, inp])
    att_out = Lambda(lambda x: tf.reduce_sum(x, axis=1))(att_out)
    
    
    y = Dense(1024, activation='relu')(att_out)
    y = BatchNormalization()(y)
    y = Dropout(0.4)(y)
    y = Dense(512, activation='relu')(y)
    y = BatchNormalization()(y)
    y = Dropout(0.4)(y)
    y = Dense(256, activation='relu')(y)
    y = BatchNormalization()(y)
    y = Dropout(0.4)(y)
    y = Dense(num_classes, activation='softmax')(y)
    
    model = Model(inputs=x, outputs=y)
    return model

In [ ]:
def get_max_pool_bilstm(embedding_matrix, num_classes, num_layers, hidden_state_size):
    
    x = Input((None,))
    embedding_layer = Embedding(
      input_dim=len(du.vocab_dict)+2, 
      output_dim=du.vec_dim,
      embeddings_initializer=Constant(embedding_matrix),
      mask_zero=True,
    )
    
    iden = Lambda(lambda x:x)
    inp = iden(x)
    inp = embedding_layer(inp)
    
    for i in range(num_layers):
        
        fw_rnn = LSTM(units=hidden_state_size, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)(inp)
        bw_rnn = LSTM(units=hidden_state_size, return_sequences=True, go_backwards=True, dropout=0.2, recurrent_dropout=0.2)(inp)
        inp = concatenate([fw_rnn, bw_rnn], axis=-1) 
    
    inp = Lambda(lambda x: tf.reduce_max(x, axis=1))(inp)
    
    
    y = Dense(1024, activation='relu')(inp)
    y = BatchNormalization()(y)
    y = Dropout(0.4)(y)
    y = Dense(512, activation='relu')(y)
    y = BatchNormalization()(y)
    y = Dropout(0.4)(y)
    y = Dense(256, activation='relu')(y)
    y = BatchNormalization()(y)
    y = Dropout(0.4)(y)
    y = Dense(num_classes, activation='softmax')(y)
    
    model = Model(inputs=x, outputs=y)
    return model

In [ ]:
train_tok_sents = list(map(word_tokenize, train_sents))
test_tok_sents = list(map(word_tokenize, test_sents))
embedding_matrix = du.create_embedding_matrix()
unk_id = du.vocab_dict['<unk>']
x_train = [[du.vocab_dict.get(w, unk_id) for w in s] for s in train_tok_sents]
x_test = [[du.vocab_dict.get(w, unk_id) for w in s] for s in test_tok_sents]
y_train = to_categorical([du.label_dict[l] for l in train_labels])
y_test = to_categorical([du.label_dict[l] for l in test_labels])

In [ ]:
train_gen = VariableLengthGenerator(x_train, y_train, BATCH_SIZE, len(du.label_dict))
valid_gen = VariableLengthGenerator(x_test, y_test, BATCH_SIZE, len(du.label_dict))

In [ ]:
# nn = get_simple_bilstm(embedding_matrix, len(du.label_dict), num_layers=1, hidden_state_size=300)
# nn = get_max_pool_bilstm(embedding_matrix, len(du.label_dict), num_layers=1, hidden_state_size=300)
nn = get_att_bilstm(embedding_matrix, len(du.label_dict), num_layers=1, hidden_state_size=300)
nn.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
nn.summary()
SVG(model_to_dot(nn).create(prog='dot', format='svg'))

In [ ]:
hist = nn.fit_generator(
    train_gen, 
    validation_data=valid_gen,
    epochs=10,
    
)

visualize_loss_and_acc(hist)

In [ ]:
nn = get_simple_bilstm(embedding_matrix, len(du.label_dict), num_layers=1, hidden_state_size=300)
# nn = get_max_pool_bilstm(embedding_matrix, len(du.label_dict), num_layers=1, hidden_state_size=300)
# nn = get_att_bilstm(embedding_matrix, len(du.label_dict), num_layers=1, hidden_state_size=300)
nn.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
hist = nn.fit_generator(
    train_gen, 
    validation_data=valid_gen,
    epochs=6,
)

visualize_loss_and_acc(hist)

# CNN method

In [ ]:
MAX_LEN = 200
train_tok_sents = list(map(word_tokenize, train_sents))
test_tok_sents = list(map(word_tokenize, test_sents))
embedding_matrix = du.create_embedding_matrix()
unk_id = du.vocab_dict['<unk>']
x_train = pad_sequences([[du.vocab_dict.get(w, unk_id) for w in s] for s in train_tok_sents],
                        padding='post', truncating='post', maxlen=MAX_LEN)
x_test = pad_sequences([[du.vocab_dict.get(w, unk_id) for w in s] for s in test_tok_sents],
                       padding='post', truncating='post', maxlen=MAX_LEN)
y_train = to_categorical([du.label_dict[l] for l in train_labels])
y_test = to_categorical([du.label_dict[l] for l in test_labels])

In [ ]:
def get_cnn(embedding_matrix, num_classes, emb_method, kernel_sizes):
  
  x = Input((MAX_LEN,))
  emb_init = None
  emb_trainable = None
  
  if emb_method=='dynamic':
    
    emb_init = Constant(embedding_matrix)
    emb_trainable = True
  elif emb_method=='static':
    
    emb_init = Constant(embedding_matrix)
    emb_trainable = False
    
  if emb_method=='rand':
    
    emb_init = 'uniform'
    emb_trainable = True
  
  embedding_layer = Embedding(
    input_dim=len(du.vocab_dict)+2, 
    output_dim=du.vec_dim,
    embeddings_initializer=emb_init,
#     mask_zero=True,
    trainable = emb_trainable
  )
  inp = embedding_layer(x)
  mask = Lambda( lambda x: K.cast(K.not_equal(x, 0), 'float32'))(x)
  expanded_mask = Lambda(lambda x: K.tile(K.expand_dims(x, axis=2), n=[1, 1, du.vec_dim]))(mask)
  inp = Multiply()([expanded_mask,  inp])

  kernel_outs = []
  for k in kernel_sizes:
    
    ker_out = Conv1D(2048, k, activation='relu')(inp)
    ker_out = Dropout(0.4)(ker_out)
    ker_out = GlobalAveragePooling1D()(ker_out)
    kernel_outs.append(ker_out)
  if len(kernel_outs)> 1:
    kernel_concat = concatenate(kernel_outs, axis=-1)
  else:
    kernel_concat = kernel_outs[0]
  y = Dense(1024, activation='relu')(kernel_concat)
  y = BatchNormalization()(y)
  y = Dropout(0.6)(y)
  y = Dense(512, activation='relu')(y)
  y = BatchNormalization()(y)
  y = Dropout(0.6)(y)
  y = Dense(128, activation='relu')(y)
  y = BatchNormalization()(y)
  y = Dropout(0.6)(y)
  y = Dense(num_classes, activation='softmax')(y)

  model = Model(inputs=x, outputs=y)
  return model
    
    
  
  

In [ ]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
nn = get_cnn(embedding_matrix, len(du.label_dict), 'dynamic', [3])
nn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
nn.summary()
SVG(model_to_dot(nn).create(prog='dot', format='svg'))

In [ ]:
hist = nn.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=256, epochs=10)
visualize_loss_and_acc(hist)

# CNN+RNN

In [ ]:
def get_rnn_cnn(embedding_matrix, num_classes, num_layers, hidden_state_size):
    
    x = Input((None,))
    embedding_layer = Embedding(
      input_dim=len(du.vocab_dict)+2, 
      output_dim=du.vec_dim,
      embeddings_initializer=Constant(embedding_matrix),
      mask_zero=False,
    )
    
    inp = embedding_layer(x)
    mask = Lambda( lambda x: K.cast(K.not_equal(x, 0), 'float32'))(x)
    expanded_mask = Lambda(lambda x: K.tile(K.expand_dims(x, axis=2), n=[1, 1, du.vec_dim]))(mask)
    inp = Multiply()([expanded_mask,  inp])
    
    for i in range(num_layers):
    
        fw_rnn = LSTM(units=hidden_state_size, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)(inp)
        bw_rnn = LSTM(units=hidden_state_size, return_sequences=True, go_backwards=True, dropout=0.2, recurrent_dropout=0.2)(inp)
        inp = concatenate([fw_rnn, bw_rnn], axis=-1) 
    
    
    
    ker_out = Conv1D(512, 2, activation='relu')(inp)
    ker_out = Dropout(0.3)(ker_out)
    
    to_dense = GlobalAveragePooling1D()(ker_out)
    
    
    y = Dense(1024, activation='relu')(to_dense)
    y = BatchNormalization()(y)
    y = Dropout(0.4)(y)
    y = Dense(512, activation='relu')(y)
    y = BatchNormalization()(y)
    y = Dropout(0.4)(y)
    y = Dense(256, activation='relu')(y)
    y = BatchNormalization()(y)
    y = Dropout(0.4)(y)
    y = Dense(num_classes, activation='softmax')(y)
    
    model = Model(inputs=x, outputs=y)
    return model

In [ ]:
nn = get_rnn_cnn(embedding_matrix, len(du.label_dict), 1, 300)
nn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
nn.summary()
SVG(model_to_dot(nn).create(prog='dot', format='svg'))

In [ ]:
hist = nn.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=256, epochs=10)
visualize_loss_and_acc(hist)

In [ ]:
def get_cnn_rnn(embedding_matrix, num_classes, emb_method, kernel_sizes, att_layer_size=[]):
  
  x = Input((MAX_LEN,))
  emb_init = None
  emb_trainable = None
  
  if emb_method=='dynamic':
    
    emb_init = Constant(embedding_matrix)
    emb_trainable = True
  elif emb_method=='static':
    
    emb_init = Constant(embedding_matrix)
    emb_trainable = False
    
  if emb_method=='rand':
    
    emb_init = 'uniform'
    emb_trainable = True
  
  embedding_layer = Embedding(
    input_dim=len(du.vocab_dict)+2, 
    output_dim=du.vec_dim,
    embeddings_initializer=emb_init,
#     mask_zero=True,
    trainable = emb_trainable
  )
  inp = embedding_layer(x)
  mask = Lambda( lambda x: K.cast(K.not_equal(x, 0), 'float32'))(x)
  expanded_mask = Lambda(lambda x: K.tile(K.expand_dims(x, axis=2), n=[1, 1, du.vec_dim]))(mask)
  inp = Multiply()([expanded_mask,  inp])

  kernel_outs = []
  for k in kernel_sizes:
    
    ker_out = Conv1D(256, k, activation='relu')(inp)
    ker_out = Dropout(0.4)(ker_out)
    ker_out = AveragePooling1D(5)(ker_out)
    kernel_outs.append(ker_out)
  if len(kernel_outs)> 1:
    kernel_concat = concatenate(kernel_outs, axis=-1)
  else:
    kernel_concat = kernel_outs[0]
    
  fw_rnn = LSTM(units=256, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)(kernel_concat)
  bw_rnn = LSTM(units=256, return_sequences=True, go_backwards=True, dropout=0.2, recurrent_dropout=0.2)(kernel_concat)
  l_out = concatenate([fw_rnn, bw_rnn], axis=-1) 
    
  alpha = l_out
  for l in att_layer_size:
    alpha = TimeDistributed(Dense(l, activation='relu'))(alpha)
  alpha = TimeDistributed(Dense(1, activation='relu'))(alpha)
  alpha = Lambda(lambda x: tf.reshape(x, [-1, tf.shape(x)[1]]))(alpha)
  alpha = Lambda(lambda x: tf.nn.softmax(x))(alpha)
  alpha = Lambda(lambda x: tf.reshape(x, [-1, tf.shape(x)[1], 1]))(alpha)
  att_out = Lambda(lambda x: tf.math.multiply(x[0], x[1]))([alpha, l_out])
  att_out = Lambda(lambda x: tf.reduce_sum(x, axis=1))(att_out)
  y = Dense(1024, activation='relu')(att_out)
  y = BatchNormalization()(y)
  y = Dropout(0.5)(y)
  y = Dense(512, activation='relu')(y)
  y = BatchNormalization()(y)
  y = Dropout(0.5)(y)
  y = Dense(128, activation='relu')(y)
  y = BatchNormalization()(y)
  y = Dropout(0.4)(y)
  y = Dense(num_classes, activation='softmax')(y)

  model = Model(inputs=x, outputs=y)
  return model

In [ ]:
nn = get_rnn_cnn(embedding_matrix, len(du.label_dict), 'dynamic', [2])
nn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
nn.summary()
SVG(model_to_dot(nn).create(prog='dot', format='svg'))


In [ ]:
hist = nn.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=256, epochs=10)
visualize_loss_and_acc(hist)